In [ ]:
!pip install -q \
  transformers accelerate bitsandbytes \
  sentence-transformers \
  langchain langchain-core langchain-community \
  langchain-huggingface \
  chromadb


In [ ]:
import pandas as pd

# Load the dataset
file_path = "/content/BBCNews.csv"   # change if your file has a different name or path
bbc_data = pd.read_csv(file_path)

# Ensure expected column names
bbc_data.columns = ["News_ID", "Description", "Tags"]

# Display basic information
print(bbc_data.head())
print(bbc_data.info())

# Cleaning and preprocessing
bbc_data["Description"] = bbc_data["Description"].fillna("")   # Handle missing descriptions
bbc_data["Tags"] = bbc_data["Tags"].fillna("")                 # Handle missing tags


In [ ]:
from langchain_core.documents import Document

# Prepare documents for ChromaDB
documents = []
for _, row in bbc_data.iterrows():
    content = row["Description"]
    tags = row["Tags"]
    meta = {"tags": tags, "news_id": row["News_ID"]}
    documents.append(Document(page_content=content, metadata=meta))

len(documents), documents[0]


In [ ]:
# Install missing splitter package (new LC version)
!pip install -q langchain-text-splitters

# Imports (new correct locations)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# ---- TEXT SPLITTING + CHROMA SETUP ----

# 1) Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " ", ""],
)

split_docs = text_splitter.split_documents(documents)
print("Number of chunks:", len(split_docs))

# 2) Embeddings model (free)
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 3) Persist directory
persist_directory = "/content/chroma_db"

# 4) Create Chroma vector store
langchain_chroma = Chroma.from_documents(
    documents=split_docs,
    embedding=hf_embeddings,
    collection_name="bbc_news",
    persist_directory=persist_directory
)

print("✅ ChromaDB initialized at:", persist_directory)


In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

# Choose model (same as tutorial)
model_id = "HuggingFaceH4/zephyr-7b-beta"

device = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"
print("Using device:", device)

# Configure model for efficient GPU memory usage (4-bit quantization)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16,
)

# Load model in quantized format
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Initialize the query pipeline
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_new_tokens=500,
    device_map="auto",
)

# Wrap into LangChain LLM
llm = HuggingFacePipeline(pipeline=query_pipeline)


In [ ]:
# Cell 5 – Build RAG chain with the modern LangChain API

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1) Make a retriever from your Chroma DB
retriever = langchain_chroma.as_retriever(search_kwargs={"k": 3})

# 2) Helper: format retrieved docs into a single context string
def format_docs(docs):
    return "\n\n".join(
        f"[news_id={d.metadata.get('news_id', '')}, tags={d.metadata.get('tags', '')}]\n{d.page_content}"
        for d in docs
    )

# 3) Define the prompt
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are an expert summarizer and analyst. Using ONLY the context provided, "
        "answer the question accurately and concisely. "
        "If the answer is not in the context, say: 'I don't know based on the dataset.'"
    ),
    (
        "human",
        "Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    ),
])

# 4) Build the RAG chain
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
# Cell 6 – Helper to ask questions and print answer + sources (fixed)

def ask_bbc(query: str):
    # 1) Get answer from the RAG chain
    answer = rag_chain.invoke(query)

    # 2) Get source documents from the retriever (new API uses .invoke)
    docs = retriever.invoke(query)   # <- this replaces get_relevant_documents()

    print(f"Question:\n{query}\n")
    print("Answer:\n", answer, "\n")

    print("Sources:")
    for idx, d in enumerate(docs, 1):
        print(f"{idx}. tags={d.metadata.get('tags', 'No Tags')} | "
              f"id={d.metadata.get('news_id', 'N/A')}")
        print(d.page_content[:200], "...\n")


# Test
ask_bbc("What does the dataset say about sports news?")
